In [3]:
import sys
import io
import requests
import json
import base64
from PIL import Image
import numpy as np
import gradio as gr
import os
import pandas as pd
import h5py
from datetime import datetime
import re
from cambrian import Dataset
import os
from gradio import Tabs


In [4]:
def iterate_h5_file(file_path,return_id):
    prompts_data = {}
    prompts_image_data = {}
    prompts_mask_data = {}
    prediction_results = {}
    answers_data = {}
    attributes = {}
    prediction_image ={}

    with h5py.File(file_path, "r") as f:
        # Extract prompts data
        prompts_group = f["prompts"]["img"]
        masks_group = f["prompts"]["mask"]
        for key in prompts_group.keys():
            image_array = np.array(prompts_group[key])
            mask_array = np.array(masks_group["mask_" + key.split("_")[1]])
            prompts_data[key] = {"image": image_array, "mask": mask_array}
            prompts_image_data[key] = Image.fromarray(image_array)
            prompts_mask_data[key] = Image.fromarray(mask_array)
            #prompts_data[key] = {"image": Image.fromarray(image_array), "mask": Image.fromarray(mask_array)}
        #prompt2 = prompts_group["image_2"]
        #mask2 = masks_group["mask_2"]
        #prompt2 = Image.fromarray(image_array)

        # Extract prediction results
        prediction_group = f["test_set"]["pred"]["pred_result_img"]
        for key in prediction_group.keys():
            prediction_array = np.array(prediction_group[key])
            prediction_results[key] = Image.fromarray(prediction_array)

        # Extract answers data
        answers_group = f["test_set"]["pred_ans"]
        for key in answers_group.keys():
            answer_array = np.array(answers_group[key])
            answers_data[key] = Image.fromarray(answer_array)

        # Extract attributes
        pred_img_group = f["test_set"]["pred"]["pred_img"]
        for key in pred_img_group.keys():
            prediction_image_array = np.array(pred_img_group[key])
            prediction_image[key] = Image.fromarray(prediction_image_array)
            attributes[key] = {}
            attributes[key]["predict_img"] = np.array(pred_img_group[key])
            if "performance" in pred_img_group[key].attrs:
                attributes[key]["performance"] = pred_img_group[key].attrs["performance"]
                print("YESSS")
            if "description" in pred_img_group[key].attrs:
                attributes[key]["description"] = pred_img_group[key].attrs["description"]
    #prompt2.show()
    if return_id=="prompts_img":
        return prompts_image_data
    if return_id=="prompts_mask":
        return prompts_mask_data
    if return_id=="prediction_results":
        return prediction_results
    if return_id=="answers_data":
        return answers_data
    if return_id=="prediction_image":
        return prediction_image
    if return_id=="performance":
        return return_id
    #return prompts_image_data, prompts_mask_data, prediction_results, answers_data, prediction_image #4,4,1,1
    #return prompts_data["image_2"], prediction_results, answers_data, prediction_image, attributes["performance"]
    #return prompt2

# Example usage
#file_path = "../file.h5"
#prompts_data, prediction_results, answers_data, prediction_image, attributes = iterate_h5_file(file_path)

    
def load_prompts_img(file_path):
    x = "prompts_img"
    prompts_data_list = iterate_h5_file(file_path,x)
    updates = [gr.Image.update(value=prompts_data_list[prompt_image], visible=True) for prompt_image in prompts_data_list]
    updates += [gr.Image.update(value=None, visible=False)] * (10 - len(prompts_data_list))
    return updates

def load_mask_img(file_path):
    x = "prompts_mask"
    prompts_mask_list = iterate_h5_file(file_path,x)
    updates = [gr.Image.update(value=prompts_mask_list[prompt_mask], visible = True) for prompt_mask in prompts_mask_list]
    updates += [gr.Image.update(value=None, visible=False)] * (10 - len(prompts_mask_list))
    return updates

def load_pred_results(file_path):
    x = "prediction_results"
    output_list = iterate_h5_file(file_path,x)
    output_list["prediction_result_img"].show()
    updates = gr.Image.update(value=output_list["prediction_result_img"], visible=True)
    
    return updates

def load_answer(file_path):
    x = "answers_data"
    output_list = iterate_h5_file(file_path,x)
    updates = gr.Image.update(value=output_list["pred_ans"], visible=True)

    return updates

def load_pred_img(file_path):
    x = "prediction_image"
    output_list = iterate_h5_file(file_path,x)
    updates = gr.Image.update(value=output_list["pred_img_1"], visible=True)

    return updates

def load_performace(file_path):
    x = "performance"
    output_list = iterate_h5_file(file_path,x)
    print(output_list)


filepath = "../file.h5"
load_performace(filepath)


performance


NameError: name 'updates' is not defined

In [ ]:
def change():
    print("debug")
    return gr.Tabs.update(selected = 'p1')
def get_path():
    path = "../file.h5"
    return path

In [ ]:
with gr.Blocks() as demo:
    file_path = gr.State('')
    demo.load(fn = get_path, outputs = file_path)
    with gr.Tabs(selected = 'p2') as maintab:
        with gr.Tab(id = 'p1',label = "page1"):
            load_button = gr.Button(value = "Enter")
            
            gr.Markdown(
                                    #title
                                    """
                                    # Page1
                                    """
                                ) 
            with gr.Row():
                prompt1 = gr.ImageMask(brush_radius=16, label = "Prompt", scale = 1)
                prompt2 = gr.ImageMask(brush_radius=16, label = "Prompt", scale = 1)
                prompt3 = gr.ImageMask(brush_radius=16, label = "Prompt", scale = 1)
                prompt4 = gr.ImageMask(brush_radius=16, label = "Prompt", scale = 1)

            
        #load_button.click(fn = iterate_h5_file, inputs = file_path, outputs = [prompt2, prediction_result, ground_truth, prediction_image, performance])
        
        with gr.Tab(id = 'p2', label = "page2"):
            gr.Markdown(
                                    #title
                                    """
                                    # Page2
                                    """
            )
            with gr.Row():
                change_button = gr.Button(value = "Enter") 
            
        change_button.click(fn = change, outputs = maintab)
        
        #change_button.click()
        #change_button.click(fn = iterate_h5_file, inputs = file_path, outputs = prompt2)
        with gr.Tab(id = 'p3', label = "page3"):
            gr.Markdown(
                                    #title
                                    """
                                    # Page3
                                    """
            )
            with gr.Row():
                button1 = gr.Button(value= "Enter")
            with gr.Row():
                with gr.Column():
                    prompt_img_list = []
                    for i in range(10):
                        prompt_image = gr.Image(label = "Prompt",visible = False)
                        prompt_img_list.append(prompt_image) 
                with gr.Column():
                    mask_img_list = []
                    for i in range(10):
                        mask_image = gr.Image(label = "Mask",visible= False)
                        mask_img_list.append(mask_image)
                with gr.Column():
                    prediction_img = gr.Image(label = "User Inputted",visible = True)
                with gr.Column():
                    prediction_result = gr.Image(label = "Result",visible = True)
                with gr.Column():
                    ground_truth = gr.Image(label = "Answer",visible = True)
                    performance = gr.Text(label = "Performance",visible = True)
                    
            button1.click(fn=load_prompts_img,inputs= file_path, outputs=prompt_img_list)        
            button1.click(fn=load_mask_img,inputs= file_path, outputs=mask_img_list)
            button1.click(fn=load_pred_img, inputs=file_path, outputs=prediction_img)
            button1.click(fn=load_answer, inputs=file_path, outputs=ground_truth)
            button1.click(fn=load_pred_results, inputs=file_path, outputs=prediction_result)


In [ ]:
demo.launch(enable_queue=True, server_name="0.0.0.0",server_port=6006)